In [6]:
!pip install openai
!pip install pandas
!pip install aiohttp
!pip install lxml


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/89/a9/63af38c7f42baff8251d937be91c6decfe9e4725fe16283dcee428e08d5c/lxml-5.3.0-cp39-cp39-macosx_10_9_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 4.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


First, create list of keywords necessary in the keyword.txt file.

Then, run command with the following command that will give top ten article information from 2022 to 2024 and save data to article_data.csv

In [8]:
!python async_pubmed_scraper.py --pages 1 --start 2018 --stop 2020 --output article_data

/Users/devammondal/PycharmProjects/geneKnowledgeGraphs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

Finding PubMed article URLs for 5 keywords found in keywords.txt

Scraping initiated for 50 article URLs found from 2018 to 2020

Preview of scraped article data:

      title     abstract  ...     keywords                                       url
0  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/31157963
1  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/31292550
2  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/31093808
3  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/29988080
4  NO_TITLE  NO_ABSTRACT  ...  NO_KEYWORDS  https://pubmed.ncbi.nlm.nih.gov/28925400

[5 rows x 8 columns]